In [ ]:
# only uses the heuristic value
# best first search algorithm
from queue import PriorityQueue

class City:
    def __init__(self, name, heuristic):
        self.name = name
        self.heuristic = heuristic
        self.neighbors = {}
        
    def add_neighbor(self, neighbor, distance):
        self.neighbors[neighbor] = distance
        # Add reverse connection too (making it bidirectional)
        neighbor.neighbors[self] = distance

def best_first_search(start, goal):
    frontier = PriorityQueue()
    frontier.put((start.heuristic, start))
    
    visited = set()
    came_from = {start: None}
    
    while not frontier.empty():
        current = frontier.get()[1]
        
        if current == goal:
            return reconstruct_path(came_from, start, goal)
            
        visited.add(current)
        
        for neighbor, _ in current.neighbors.items():
            if neighbor not in visited:
                frontier.put((neighbor.heuristic, neighbor))
                came_from[neighbor] = current
    
    return None

def reconstruct_path(came_from, start, goal):
    current = goal
    path = []
    
    while current != start:
        path.append(current.name)
        current = came_from[current]
    
    path.append(start.name)
    path.reverse()
    return path

# Example usage
if __name__ == "__main__":
    # Create cities
    london = City("London", 0)
    paris = City("Paris", 344)
    brussels = City("Brussels", 322)
    amsterdam = City("Amsterdam", 358)
    berlin = City("Berlin", 932)
    
    # Add connections (only need to add one direction now)
    london.add_neighbor(paris, 344)
    london.add_neighbor(brussels, 373)
    paris.add_neighbor(brussels, 265)
    paris.add_neighbor(berlin, 1054)
    brussels.add_neighbor(amsterdam, 173)
    brussels.add_neighbor(berlin, 746)
    amsterdam.add_neighbor(berlin, 577)
    
    # Find path from Berlin to London
    path = best_first_search(berlin, london)
    
    # Add error handling for path output
    if path:
        print(f"Best First Search Path: {' -> '.join(path)}")
        
        # Calculate total distance
        def calculate_total_distance(path_cities):
            total_distance = 0
            for i in range(len(path_cities)-1):
                city1_name = path_cities[i].lower()
                city2_name = path_cities[i+1].lower()
                city1 = eval(city1_name)
                city2 = eval(city2_name)
                total_distance += city1.neighbors[city2]
            return total_distance
        
        total_distance = calculate_total_distance(path)
        print(f"Total Distance: {total_distance} km")
    else:
        print("No path found between Berlin and London!")

    # Print available connections for debugging
    print("\nAvailable connections:")
    for city in [london, paris, brussels, amsterdam, berlin]:
        print(f"\n{city.name} connected to:")
        for neighbor, distance in city.neighbors.items():
            print(f"  - {neighbor.name}: {distance} km")

Best First Search Path: Berlin -> Brussels -> London
Total Distance: 1119 km

Available connections:

London connected to:
  - Paris: 344 km
  - Brussels: 373 km

Paris connected to:
  - London: 344 km
  - Brussels: 265 km
  - Berlin: 1054 km

Brussels connected to:
  - London: 373 km
  - Paris: 265 km
  - Amsterdam: 173 km
  - Berlin: 746 km

Amsterdam connected to:
  - Brussels: 173 km
  - Berlin: 577 km

Berlin connected to:
  - Paris: 1054 km
  - Brussels: 746 km
  - Amsterdam: 577 km


In [ ]:
# use both the path cost (g) and heuristic (h) to calculate total cost f = g + h.
# A* algorithm
class Location:
    def __init__(self, name, distance_to_destination):
        self.name = name
        self.distance = distance_to_destination  # Straight line distance to destination
        self.neighbors = {}  # Dictionary of neighbor locations and road distances
        self.traffic_level = 1  # 1: Clear, 2: Moderate, 3: Heavy
    
    def add_neighbor(self, neighbor, road_distance):
        self.neighbors[neighbor] = road_distance
        # Make connection bidirectional
        neighbor.neighbors[self] = road_distance

def calculate_travel_time(distance, traffic_level):
    # Assume average speed: 60km/h with clear traffic
    # Traffic levels affect speed: Clear = 1x, Moderate = 1.5x, Heavy = 2x
    base_time = distance / 60  # Time in hours
    return base_time * traffic_level

def a_star_navigation(start, destination):
    open_set = {start}  # Locations to be evaluated
    closed_set = set()  # Already evaluated locations
    
    # Store g_scores (time from start) and f_scores (estimated total time)
    g_score = {start: 0}
    f_score = {start: start.distance}
    
    # Store the path
    came_from = {}
    
    while open_set:
        # Find location with lowest f_score
        current = min(open_set, key=lambda loc: f_score.get(loc, float('inf')))
        
        if current == destination:
            return reconstruct_path(came_from, current)
        
        open_set.remove(current)
        closed_set.add(current)
        
        # Check all neighbors
        for neighbor, road_distance in current.neighbors.items():
            if neighbor in closed_set:
                continue
            
            # Calculate time including traffic
            travel_time = calculate_travel_time(road_distance, neighbor.traffic_level)
            tentative_g_score = g_score[current] + travel_time
            
            if neighbor not in open_set:
                open_set.add(neighbor)
            elif tentative_g_score >= g_score.get(neighbor, float('inf')):
                continue
            
            # This path is the best so far
            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = g_score[neighbor] + neighbor.distance

    return None

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

# Example usage
def main():
    # Create locations with distances to final destination (in km)
    home = Location("Home", 15)
    market = Location("Market", 12)
    school = Location("School", 8)
    park = Location("Park", 5)
    mall = Location("Mall", 3)
    office = Location("Office", 0)  # Destination
    
    # Add road connections with distances (in km)
    home.add_neighbor(market, 5)
    home.add_neighbor(school, 7)
    market.add_neighbor(park, 6)
    school.add_neighbor(park, 4)
    park.add_neighbor(mall, 3)
    mall.add_neighbor(office, 3)
    
    # Add some traffic conditions
    market.traffic_level = 2  # Moderate traffic
    park.traffic_level = 3    # Heavy traffic
    
    # Find best route
    route = a_star_navigation(home, office)
    
    # Print the route
    print("Best route to Office:")
    if route:
        total_distance = 0
        total_time = 0
        
        for i in range(len(route)-1):
            current = route[i]
            next_loc = route[i+1]
            distance = current.neighbors[next_loc]
            time = calculate_travel_time(distance, next_loc.traffic_level)
            
            total_distance += distance
            total_time += time
            
            print(f"{current.name} -> {next_loc.name}")
            print(f"  Distance: {distance}km")
            print(f"  Traffic Level: {'Clear' if next_loc.traffic_level == 1 else 'Moderate' if next_loc.traffic_level == 2 else 'Heavy'}")
            print(f"  Estimated Time: {time:.1f} hours")
            print()
        
        print(f"Total Distance: {total_distance}km")
        print(f"Total Estimated Time: {total_time:.1f} hours")
    else:
        print("No route found!")

if __name__ == "__main__":
    main()